In [9]:
#전부 리스트에 담는다 조회리스트 [제목, 링크, 종류] (해쉬로 담아야겠다)
import requests
from bs4 import BeautifulSoup
import telegram
import time
import re
#텔레그램 토큰   
#토큰
token = ''
message_token = 1829234297
bot = telegram.Bot(token = token)
def message(title, link, shop, shop_link):
    link, shop, shop_link = str(link), str(shop), str(shop_link)
    tt = f'''{title}\n\n{shop}_link : \n{link}\n\nprotuct_link : \n{shop_link}
    '''
    return tt

send = []

while True:
    ppompu_num = 0
    try:
        check = []

        #뽐뿌
        ppompu = requests.get('https://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu')
        ppompu = ppompu.text
        ppompu_soup = BeautifulSoup(ppompu,'html.parser')
        for tag in ppompu_soup.find_all('td', class_ = 'list_vspace',align = 'left'):
            if tag.find('td', valign = 'middle') == None: continue
            ppompu_num += 1
            parent = tag.find('td', valign = 'middle')
            title = parent.find('font').get_text()
            link = parent.find('a')['href']
            #없을때는 예외 처리 
            try:
                comment = parent.find('span', class_ = 'list_comment2').get_text()
            except:
                comment = ''
            #[title, link, kind]
            #키워드가 존재하면 
            check.append([title, link, 'ppompu',' 댓'+comment ])
            if ppompu_num == 21: break
        #뽐뿌 인기
        ppompu = requests.get('https://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu&hotlist_flag=999')
        ppompu = ppompu.text
        ppompu_soup = BeautifulSoup(ppompu,'html.parser')
        for tag in ppompu_soup.find_all('td', class_ = 'list_vspace',align = 'left'):
            if tag.find('td', valign = 'middle') == None: continue
            ppompu_num += 1
            parent = tag.find('td', valign = 'middle')
            title = parent.find('font').get_text()
            link = parent.find('a')['href']
            #없을때는 예외 처리 
            try:
                comment = parent.find('span', class_ = 'list_comment2').get_text()
            except:
                comment = ''
            #[title, link, kind]
            #키워드가 존재하면 
            check.append([title + 'ppompu_ingi', link, 'ppompu',' 댓'+comment ])
            if ppompu_num == 21: break
        

        #펨코 https://www.fmkorea.com/hotdeal
        fmkorea = requests.get('https://www.fmkorea.com/hotdeal')
        fmkorea = fmkorea.text
        fmkorea_soup = BeautifulSoup(fmkorea, 'html.parser')
        for tag in fmkorea_soup.find('div', class_ = 'fm_best_widget _bd_pc').find('ul').find_all('li'):
            #제목
            title = tag.find('h3').get_text().strip().split()
            if '[' in title[-1]:
                title = ' '.join(title[:-1])
            elif '포텐' in title[-1]:
                #댓글 없이 포텐
                if '[' in title[-2]:
                    title = ' '.join(title[:-2])
                else:
                    title = ' '.join(title[-1])
            else:
                title = ' '.join(title)
            price = tag.find('div', class_ = 'hotdeal_info').find_all('span')[1].get_text().strip()
            link = tag.find('a')['href']
            comment = re.findall('\[([^)])+]', tag.find('h3').get_text().strip())
            #댓글 처리
            if comment == []: comment = 0
            else: comment = comment[0]

            #포텐이 아닌거 예외 처리 
            try:
                poten = '[' + tag.find('span', title = '베스트').get_text() + ']'
            except:
                poten = ''
            check.append([title + price + poten, link, 'fmkorea', comment])

        #쿨엔조이 https://coolenjoy.net/bbs/jirum
        cool = requests.get('https://coolenjoy.net/bbs/jirum')
        cool = cool.text
        cool_soup = BeautifulSoup(cool, 'html.parser')

        for tag in cool_soup.find_all('td', class_ = 'td_subject'):
            title = tag.find('a').get_text().strip().split('댓글')[0].strip()
            try:
                comment = tag.find('a').find('span', class_ = 'cnt_cmt').get_text()
            except:
                comment = str(0)
            link = tag.find('a')['href']
            check.append([title, link, 'coolnjoy',' 댓'+ comment])


        #봇키는데 렛걸리나..

        #키워드가 안에 존재하면 이미 보낸거는 보내기 리스트에 담는다 
        keyword = ['[포텐]','ppompu_ingi']

        #<=========================================================>
        #check 모든 title 한 최대 100개 정도 리스트 (종류별로))
        #check list : 모든 title값만
        #send 한번 텔레그램에 보낸것들 예외처리
        check_lst = []
        for item in check:
            title = item[0]
            check_lst.append(title)
            link = item[1]
            where = item[2]
            comment = item[3]
            if title in send: continue
            for key in keyword:
                if key in title :
                    if where == 'ppompu':
                        ppompu_html = requests.get('https://www.ppomppu.co.kr/zboard/' + link)
                        ppompu_html = ppompu_html.text
                        ppompu_html_soup = BeautifulSoup(ppompu_html, 'html.parser')
                        site_link = 'https://www.ppomppu.co.kr/zboard/' + link
                        try:
                            shop_link = ppompu_html_soup.find('div', 'sub-top-text-box').find('div',class_ = 'wordfix').find('a', target = '_blank').get_text()
                        except:
                            shop_link = ''
                    elif where == 'fmkorea':
                        fmkorea_html = requests.get('https://www.fmkorea.com' + link)
                        fmkorea_html = fmkorea_html.text
                        fmkorea_html_soup = BeautifulSoup(fmkorea_html, 'html.parser')
                        site_link = 'https://www.fmkorea.com' + link
                        try:
                            shop_link = fmkorea_html_soup.find('div', class_ = 'xe_content').find('a')['href']
                        except:
                            shop_link = ''
                    else:
                        cool_html = requests.get(link)
                        cool_html = cool_html.text
                        cool_html_soup = BeautifulSoup(cool_html, 'html.parser')
                        try:
                            shop_link = cool_html_soup.select_one('#bo_v_link').find('strong').get_text()
                        except:
                            shop_link = ''
                        site_link = link
                    #보낸것들은 따로 저장
                    send.append(title)
                    comment = str(comment)
                    message_ = message(title + comment, site_link, where, shop_link)
                    try:
                        bot.send_message(message_token, message_)
                    except:
                        time.sleep(10)
                        bot.send_message(message_token, message_)
        #체크 리스트에 없으면 삭제 

        #조회리스트에 보내기 리스트가 없으면 삭제 
        for send_item in send:
            if send_item not in check_lst:
                send.remove(send_item)
        time.sleep(60)
    except:
        time.sleep(10)
        continue
    

/var/folders/4h/pt3p68k541577dsnm7h_t8dh0000gn/T/ipykernel_26851/424074918.py:162: RuntimeWarning: coroutine 'Bot.send_message' was never awaited
  bot.send_message(message_token, message_)


In [1]:
import requests
from bs4 import BeautifulSoup
# import telegram

check = []
send = []
#뽐뿌
ppompu = requests.get('https://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu')
ppompu = ppompu.text
ppompu_soup = BeautifulSoup(ppompu,'html.parser')
for tag in ppompu_soup.find_all('td', class_ = 'list_vspace',align = 'left'):
    if tag.find('td', valign = 'middle') == None: continue
    # 이미지가 있으면 예외처리
    parent = tag.find('td', valign = 'middle')
    title = parent.find('font').get_text()
    link = parent.find('a')['href']
    #없을때는 예외 처리 
    try:
        comment = parent.find('span', class_ = 'list_comment2').get_text()
    except:
        comment = ''
    #[title, link, kind]
    #키워드가 존재하면 
    check.append([title+comment, link, 'ppompu'])
    


In [119]:
ppompu_html = requests.get('https://www.ppomppu.co.kr/zboard/' + 'view.php?id=ppomppu&page=1&divpage=71&no=417003')
ppompu_html = ppompu_html.text
ppompu_html_soup = BeautifulSoup(ppompu_html, 'html.parser')
print(ppompu_html_soup.find('div', 'sub-top-text-box').find('div',class_ = 'wordfix').find('a', target = '_blank').get_text())


https://www.coupang.com/vp/products/4777658786?itemId=6105494299&vendorItemId=4514067748&isAddedCart=


In [5]:
#펨코 https://www.fmkorea.com/hotdeal
import requests
from bs4 import BeautifulSoup

fmkorea = requests.get('https://www.fmkorea.com/hotdeal')
fmkorea = fmkorea.text
fmkorea_soup = BeautifulSoup(fmkorea, 'html.parser')
for tag in fmkorea_soup.find('div', class_ = 'fm_best_widget _bd_pc').find('ul').find_all('li'):
    #제목
    title = tag.find('h3').get_text().strip().split()
    price = tag.find('div', class_ = 'hotdeal_info').find_all('span')[1].get_text().strip()
    link = tag.find('a')['href']
    #포텐이 아닌거 예외 처리 
    try:
        poten = '[' + tag.find('span', title = '베스트').get_text() + ']'
    except:
        poten = ''
    print(title, price, link, poten)

['면도기', '+', '면도기날', '총11입', '+', '폼74ml', '+', '페이스6스타일', '면도기', '증정'] 가격: 20,320원 /5517097471 
['갤주를', '넘은', '홈페이지', '주인', 'FM2023', '[10]'] 가격: 48,000원 /5517065981 
['kfc', '13%', '잔액권', '[14]'] 가격: 17,400원 /5516889065 
['프로스펙스', '스펙터', '103/102', '[81]'] 가격: 24,510원 /5516862273 
['리얼', '그래놀라400g', '3봉', '+', '증정3개', '[23]', '포텐'] 가격: 12,990원 /5516839859 [포텐]
['구구바', '아이스크림', '30개입', '[44]'] 가격: 17,610원 /5516835882 
['HP', '파빌리온', '14-ec1020AU', '/', 'R7', '바르셀로', '/', '8GB', '/', '256GB', '/', 'FreeDos', '[46]', '포텐'] 가격: 554,070원 /5516805589 [포텐]
['듀이트리', '병풀', '100', '마스크', '20매', '[22]'] 가격: 10,240원 /5516574031 
['삼성', '980pro', '2TB', '[31]'] 가격: $159.99 /5516544643 
['정림전자', '27인치', 'QHD', 'JE271L7711', '[32]'] 가격: 128,250원 /5516542357 
['티몬', '단하루', '해남베니하루카', '특상3키로', '[1]'] 가격: 9,600원 /5516503716 
['리코셀', '알로에', '수딩젤', '대용량(1L', 'x2)', '[20]'] 가격: (쿠팡와우)6,250 /5516483256 
['농심', '올리브', '짜파게티', '25개입', '[92]'] 가격: 15,500원 /5516458968 
['라미', '알스타', 's펜(브론즈)', '[56]'] 가격: 59,

In [124]:
fmkorea_html = requests.get('https://www.fmkorea.com' + '/4518630434')
fmkorea_html = fmkorea_html.text
fmkorea_html_soup = BeautifulSoup(fmkorea_html, 'html.parser')
print(fmkorea_html_soup.find('div', class_ = 'xe_content').find('a')['href'])


http://www.11st.co.kr/products/3533225681/share?


In [146]:
#쿨엔조이 https://coolenjoy.net/bbs/jirum

cool = requests.get('https://coolenjoy.net/bbs/jirum/p18?sfl=wr_subject&sop=and')
cool = cool.text
cool_soup = BeautifulSoup(cool, 'html.parser')

for tag in cool_soup.find_all('td', class_ = 'td_subject'):
    title = tag.find('a').get_text().strip().split('댓글')[0].strip()
    try:
        comment = tag.find('a').find('span', class_ = 'cnt_cmt').get_text()
    except:
        comment = 0
    link = tag.find('a')['href']
    check.append(title + commnet, link, 'coolnjoy')
    print(title,comment, link)


#텔레그램 토큰

[롯데온]ASUS TUF H670-PRO WIFI D4 220,410원/2500원 ＊종료됨 [8] https://coolenjoy.net:443/bbs/jirum/1629213?sfl=wr_subject&sop=and&page=18
[네이버페이] 적립 10원, 종합 목록 (22.03.05) ＊종료됨 [22] https://coolenjoy.net:443/bbs/jirum/1629204?sfl=wr_subject&sop=and&page=18
[포게임] 봄맞이 프로모션 (다양/무료) 0 https://coolenjoy.net:443/bbs/jirum/1629203?sfl=wr_subject&sop=and&page=18
[리더스/11번가] RTX3070Ti FE / 3080Ti FE 추가물량… [34] https://coolenjoy.net:443/bbs/jirum/1629176?sfl=wr_subject&sop=and&page=18
펑 [11] https://coolenjoy.net:443/bbs/jirum/1629159?sfl=wr_subject&sop=and&page=18
[Newegg] WD RED PLUS 8TB WD80EFBX ($149.99/미국내… [4] https://coolenjoy.net:443/bbs/jirum/1629154?sfl=wr_subject&sop=and&page=18
[에픽게임즈] 센티피드: 리차지드 무료 배포 시작 ＊종료됨 [17] https://coolenjoy.net:443/bbs/jirum/1629121?sfl=wr_subject&sop=and&page=18
[에픽게임즈] 블랙 위도우: 리차지드 무료 배포 시작 ＊종료됨 [14] https://coolenjoy.net:443/bbs/jirum/1629120?sfl=wr_subject&sop=and&page=18
[네이버페이] 적립 10원, 종합 목록 (22.03.04) ＊종료됨 [21] https://coolenjoy.net:443/bbs/jirum/1629108?sfl=wr

In [3]:
import requests
from bs4 import BeautifulSoup
ppompu = requests.get('https://www.ppomppu.co.kr/zboard/zboard.php?id=ppomppu&hotlist_flag=999')
ppompu = ppompu.text
ppompu_soup = BeautifulSoup(ppompu,'html.parser')
ppompu_num = 0
for tag in ppompu_soup.find_all('td', class_ = 'list_vspace',align = 'left'):
    if tag.find('td', valign = 'middle') == None: continue
    ppompu_num += 1
    parent = tag.find('td', valign = 'middle')
    title = parent.find('font').get_text()
    link = parent.find('a')['href']
    #없을때는 예외 처리 
    try:
        comment = parent.find('span', class_ = 'list_comment2').get_text()
    except:
        comment = ''
    print(title)
    #[title, link, kind]
    #키워드가 존재하면 
    if ppompu_num == 20: break

[G마켓] CJ제일제당 맛밤50G X 40개 (37100원/무배)
[지마켓] 농협라이블리 한돈 삼겹살 구이용 500g+목살 구이용 500g (14,900/무료)
[네이버] 썬키스트 블랙라벨 오렌지 2.4kg (9,900원/무료)
[옥션] 붕어싸만코 30개 (20,800원/무료)
[지마켓] 멸균우유,함박스테이크,라면,볶음밥,연어구이,돈까스,떡볶이,고등어,두유,화장지,쌀,햇반,컵밥 등 (다양/무료)
[쿠팡] 프리메라 맨 오가니언스 스페셜 2종 세트 (45,600원/와우 35,600원-적립15,360원/와우무배)
[핵이득마켓] 임박상품주의 이베리코 베요타 훈연 폴드포크 스테이크(돼지목살) 150g 1+1(=300g) (3,500원/3,500원)
[G마켓] 닥터스베스트 킬레이트화 마그네슘 240정 2개 (48,030/무료)
[11마존] Crucial MX500 3D NAND 내장 SSD 1TB SATA (71,910/우주패스 무료)
[쿠팡] 농심 올리브 짜파게티 25입 (15,500원/무료)
[홈플러스] 보리먹인돼지 항정살 1팩 600g (13,000원/3,000원)
[티몬] 갤럭시탭s8 128gb 외 (699,170원/무료)
[G마켓] 펩시제로 210ml x 30캔 (13,730/무료)
[11번가] 오뚜기밥 210g 24개입 (20,900원/무료)
[네이버] 책가방 가방 (5,900원/무료)
[11마존] Tempur-Pedic 템퍼 클라우드 베개 화이트 스탠다드 (72,870원/우주패스무료)
[G마켓/옥션] 아몬드 푸레이크 1200g (8900/3000) 위메프 쿡솜씨 해물오꼬노미야끼 6630원 외
[11번가] 뚜레쥬르 데일리 우유식빵 (2,000원/무료)
[옥션] 올반 꿔바로우 (24730원/무배)
[티몬] HACCP인증 구운란 대란 60구 (11,920원/무료)


In [142]:
cool_html = requests.get('https://coolenjoy.net:443/bbs/jirum/1639984')
cool_html = cool_html.text
cool_html_soup = BeautifulSoup(cool_html, 'html.parser')
print(cool_html_soup.select_one('#bo_v_link').find('strong').get_text())

https://www.newegg.com/team-32gb-288-pin-ddr5-sdram/p/N82E16820331746?…
